# Train & Align NN Acoustic Model
Align already while training.

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
%run ../prongen/hmm_pron.py --in-jupyter
%run ../acmodel/plot.py
%matplotlib ipympl

%run ../acmodel/matrix.py
%run ../acmodel/praat_ifc.py

%run ../acmodel/hmm_acmodel.py

%run ../acmodel/nn_acmodel.py

## Get training data
We aligned Czech CommonVoice train set using an ultra-prinmitive HMM/GMM. Let's use it as a starting point.

In [ ]:
#import pandas as pd
#pd.set_option('display.max_colwidth', None)

#f = pd.read_csv("nn_train.tsv", sep="\t", keep_default_na=False)
#df = pd.read_csv("nn_train_g2.tsv", sep="\t", keep_default_na=False)
#df = pd.read_csv("nn_train_g3.tsv", sep="\t", keep_default_na=False)
#df = pd.read_csv("nn_train_g4.tsv", sep="\t", keep_default_na=False)


In [ ]:
#df

In [ ]:
#hmms = []
#for wav, sentence, targets in list(zip(df.wav.values, df.sentence.values, df.targets.values)):
#    hmm = HMM(sentence, wav=wav, use_DA=True)
#    hmm.targets = targets
#    hmms.append(hmm)
    

df = pd.read_csv("nn_train_g5.tsv", sep="\t", keep_default_na=False)  # USED JUST AS A TEMPLATE FOR SAVED ALIGNMENTS
    
    
hmms = get_training_hmms("nn_train_g5.tsv", derivatives=3)

In [ ]:
#b_set = sorted({*"".join([hmm.b for hmm in hmms ])}) # make sorted set of all phone names in the training set

all_mfcc, all_targets, b_set = collect_training_material(hmms)

out_size = len(b_set)
in_size = hmms[0].mfcc.size(1)



" ".join(b_set), out_size, in_size

In [ ]:
#all_targets = "".join([hmm.targets for hmm in hmms])
#train_len = len(all_targets)

In [ ]:
#all_mfcc = torch.cat([hmm.mfcc for hmm in hmms]).double().to(device)
#assert all_mfcc.size()[0]==train_len

## Setup PyTorch training tools

In [ ]:
##import os
#import torch
#from torch import nn
#from torch.utils.data import DataLoader
#from torch.utils.data import Dataset
##from torchvision import datasets, transforms

In [ ]:
model = NeuralNetwork(in_size, out_size).to(device)
print(model)

In [ ]:
from torch.utils.data import DataLoader

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#torch.save(model.state_dict(), 'model_weights_g4_20.pth')

In [ ]:
training_data = SpeechDataset(all_mfcc, all_targets, b_set) # initial alignment


for mega_epoch in range(100): # change to 100
    print(f"============ Training Mega Epoch {mega_epoch} =============")

    all_targets = "".join([hmm.targets for hmm in hmms])  # collect alignments
    training_data.all_targets = all_targets  # just update the object with new targets

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True) # new dataloader for this alignment

    train_n_epochs(train_dataloader, optimizer, model, criterion, 5) # change to 10

    print('Interupted training for re-alignment...')

    model.eval() # switch to evaluation mode


    for idx, hmm in enumerate(hmms):
        if idx%1000==0:
            print(f"Align {idx}")
    
        alpha_m, alpha_exp = viterbi_align_nn(hmm, model, b_set)
        alp = torch.cat(alpha_m.val)
        hmm.intervals, bap = backward_alignment_pass_intervals(hmm, alp)
        hmm.indices = i = bap.max(1).indices
        s = "".join([hmm.b[ii] for ii in i])
        hmm.troubling = troubling_alignmet(s)
        hmm.targets = "".join([hmm.b[ii] for ii in i])
        
    df['targets'] = [hmm.targets for hmm in hmms]

    filename_base = f"mega_training_{'%04d' % mega_epoch}"

    torch.save(model.state_dict(), filename_base+".pth")
    df.to_csv(filename_base+".tsv", sep="\t", index=False)



In [ ]:
device

In [ ]:
STOP

https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html